In [190]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

import pyrr 
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import itertools
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
def cuboid_data2(o, size=(1,1,1)):
    X = [[[0, 1, 0], [0, 0, 0], [1, 0, 0], [1, 1, 0]],
         [[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]],
         [[1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
         [[0, 0, 1], [0, 0, 0], [0, 1, 0], [0, 1, 1]],
         [[0, 1, 0], [0, 1, 1], [1, 1, 1], [1, 1, 0]],
         [[0, 1, 1], [0, 0, 1], [1, 0, 1], [1, 1, 1]]]
    X = np.array(X).astype(float)
    for i in range(3):
        X[:,:,i] *= size[i]
    X += np.array(o)
    return X

def plotCubeAt2(positions,sizes=None,colors=None, **kwargs):
    if not isinstance(colors,(list,np.ndarray)): 
        colors=["C0"]*len(positions)
    if not isinstance(sizes,(list,np.ndarray)): sizes=[(1,1,1)]*len(positions)
    g = []
    # colors = np.empty(axes + [4], dtype=np.float32)
    alpha = 0.2
    # colors[:] = [1, 0, 0, alpha]  # red
    for p,s,c in zip(positions,sizes,colors):
        g.append( cuboid_data2(p, size=s) )
    return Poly3DCollection(np.concatenate(g),  
                            facecolors=(colors), edgecolor="k")

def split_BB(box, voxel_size):

    minBB, maxBB = box
    minBB = np.array(minBB)
    maxBB = np.array(maxBB)

    pr_invoxels = (maxBB - minBB)/voxel_size
    # check that dimensions of plant region are  multiples of the voxel size
    if np.all(np.mod(pr_invoxels, 1) != 0):
        raise ValueError('Voxel size %.3f is not a multiple of the plant region dimensions' %(voxel_size))

    # Split each side of plant region by 2 but get integers only
    mid_distances = np.floor_divide(pr_invoxels,2)
    xi, yi, zi = np.transpose([[0,0,0], mid_distances.tolist()])
    xf, yf, zf = np.transpose([mid_distances.tolist(), pr_invoxels.tolist()])

    minBB_ = list(itertools.product(xi, yi, zi))
    maxBB_ = list(itertools.product(xf, yf, zf))

    boxes = []
    for i in range(len(minBB_)):

        A = np.array(minBB) + np.array(minBB_[i]) * voxel_size
        B = np.array(minBB) + np.array(maxBB_[i]) * voxel_size
        boxes.append(pyrr.aabb.create_from_bounds(A, B).tolist())

    return boxes

In [208]:
voxel_size = 0.2

# Plot figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

mockname = 'single_57'
mockdir = os.path.join(_data, mockname)
spos = os.path.join(mockdir, 'scanner_pos.txt')
rawdata_files = glob.glob(os.path.join(mockdir, 's1*.npy'))

scan = lad.laod_scan_pos(spos)
id = [i.decode("utf-8") for i in scan['scan']]

# get bounding box of plant region
segtrees_dir = os.path.join(mockdir, 'toy_trees')
segtrees_files = glob.glob(os.path.join(segtrees_dir, 'tree_*.npy'))
tree = np.load(segtrees_files[0])
pointsPR = tree.T[5:8].T
pcd = lad.points2pcd(pointsPR)
voxPR = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
maxBB = voxPR.get_max_bound()
minBB = voxPR.get_min_bound()
width, height, depth = maxBB - minBB

# plot bounding box
positions = [(minBB[0], minBB[1], minBB[2])]
sizes = [(width, height, depth)]
pc = plotCubeAt2(positions,sizes,colors=[1,0,0,0.2], edgecolor="k")
ax.add_collection3d(pc)

# create BB for ray interceptio
boxPR = pyrr.aabb.create_from_bounds(minBB, maxBB)

print(positions)
print(sizes)

ax.set_xlim([-18,18])
ax.set_ylim([-18,18])
ax.set_zlim([0,20])

for file in rawdata_files:

    filename = file.split('/')[-1]

    df = np.load(file)
    sample = 3
    keep = np.random.randint(0, len(df), sample)
    df = df[keep]

    points = df.T[5:8].T
    keep = np.array(id) == filename[:2]
    _, sx, sy, sz = scan[keep][0]
    # sensor coordinates
    p2 = [sx, sy, sz]
    ax.scatter3D(sx, sy, sz, c='r', s=20, marker='*')

    for p1 in points:

        line = pyrr.line.create_from_points(p1, p2, dtype=None)
        ray = pyrr.ray.create_from_line(line)

        res = pyrr.geometric_tests.ray_intersect_aabb(ray, boxPR)
        ax.scatter3D(*res, c='k', s=10)

        if res is not None:

            boxes_reached = split_BB(boxPR, voxel_size)
            # boxes_reached = []
            for i in range(5):
                boxes = []
                for box in boxes_reached:
                    res = pyrr.geometric_tests.ray_intersect_aabb(ray, box)
                    if res is not None:
                        boxes.append(box)
                boxes_reached = []
                for box in boxes:
                    # print(box)
                    boxes_ = split_BB(box, voxel_size)
                    for j in boxes_:
                        boxes_reached.append(j)

        for box in boxes:

            minBB, maxBB = box
            minBB = np.array(minBB)
            maxBB = np.array(maxBB)
            width, height, depth = maxBB - minBB
            # plot bounding box
            positions = [(minBB[0], minBB[1], minBB[2])]
            sizes = [(width, height, depth)]
            pc = plotCubeAt2(positions,sizes,colors=[0,1,0,0.4], edgecolor="k")
            ax.add_collection3d(pc)

        ax.plot(*line.T.tolist())
        ax.scatter3D(*p1, c='g', s=10)


[(-4.986403083801269, -4.757557010650634, 3.8480516910552978)]
[(9.600000000000001, 9.8, 6.0)]


ValueError: Voxel size 0.200 is not a multiple of the plant region dimensions

In [196]:
voxel_size=0.5

# get bounding box of plant region
segtrees_dir = os.path.join(mockdir, 'toy_trees')
segtrees_files = glob.glob(os.path.join(segtrees_dir, 'tree_*.npy'))
tree = np.load(segtrees_files[0])
pointsPR = tree.T[5:8].T
pcd = lad.points2pcd(pointsPR)
voxPR = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
maxBB = voxPR.get_max_bound()
minBB = voxPR.get_min_bound()
width, height, depth = maxBB - minBB


print(type(minBB))
box = [minBB, maxBB]
voxel_size = 0.5
boxes = split_BB(box, voxel_size)
boxes


<class 'numpy.ndarray'>


[[[-5.1364030838012695, -4.907557010650635, 3.698051691055298],
  [-0.13640308380126953, 0.09244298934936523, 6.698051691055298]],
 [[-5.1364030838012695, -4.907557010650635, 6.698051691055298],
  [-0.13640308380126953, 0.09244298934936523, 10.198051691055298]],
 [[-5.1364030838012695, 0.09244298934936523, 3.698051691055298],
  [-0.13640308380126953, 5.092442989349365, 6.698051691055298]],
 [[-5.1364030838012695, 0.09244298934936523, 6.698051691055298],
  [-0.13640308380126953, 5.092442989349365, 10.198051691055298]],
 [[-0.13640308380126953, -4.907557010650635, 3.698051691055298],
  [4.8635969161987305, 0.09244298934936523, 6.698051691055298]],
 [[-0.13640308380126953, -4.907557010650635, 6.698051691055298],
  [4.8635969161987305, 0.09244298934936523, 10.198051691055298]],
 [[-0.13640308380126953, 0.09244298934936523, 3.698051691055298],
  [4.8635969161987305, 5.092442989349365, 6.698051691055298]],
 [[-0.13640308380126953, 0.09244298934936523, 6.698051691055298],
  [4.863596916198730

In [93]:

    

positions = [(-3,5,-2),(1,7,1)]
sizes = [(4,5,3), (3,3,7)]
colors = ["crimson","limegreen"]

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect('auto')

pc = plotCubeAt2(positions,sizes,colors=None, edgecolor="k")
ax.add_collection3d(pc)    

ax.set_xlim([-4,6])
ax.set_ylim([4,13])
ax.set_zlim([-3,9])

plt.show()

<ipython-input-93-c91f7bf873af>:33: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')
